In [1]:
import os

import gt_extras as gte
import polars as pl
import polars.selectors as cs
import polarspiper as ppp
from great_tables import (
    GT,
    exibble,
    loc,
    md,
    google_font,
    nanoplot_options,
    style,
    system_fonts,
)

from greatest_running_table.create_parquet import create_parquet

In [2]:
parquet_file = "../data/activities.parquet"
if not os.path.exists(parquet_file):
    create_parquet(destination=parquet_file)

In [17]:
df = (
    pl.scan_parquet(parquet_file)
    .select(cs.exclude("stream"))
    .collect()
    .with_columns(
        pl.from_epoch("time"),
        pl.from_epoch("created"),
        pl.from_epoch("edited"),
    )
    .pipe(ppp.drop_columns_that_are_all_null)
    # .with_columns(pl.col("zoneDistributionHr").fill_null(pl.lit([0] * 5)))
    # .with_columns(pl.col("zoneDistributionPace").fill_null(pl.lit([0] * 9)))
    .with_columns(
        date=pl.col("time").dt.date(),
    )
    .with_columns(sport=pl.col("sport").rank("dense"))
    .with_columns(
        duration=pl.col("duration").map_elements(
            lambda s: (f"{int(s // 3600)}h" if s >= 3600 else "")
            + (f"{int((s % 3600) // 60)}m" if s >= 60 else "")
            + (f"{int(s % 60)}s" if s < 60 or int(s % 60) > 0 else ""),
            return_dtype=pl.Utf8,
        )
    )
    .select(
        "date",
        "time",
        "sport",
        "title",
        "distance",
        "duration",
        "elevationUp",
        "elevationDown",
        "hrAvg",
        "hrMax",
        "kcal",
        # "vo2max",
        "rpe",
        "zoneDistributionPace",
        "zoneDistributionHr",
    )
    .sort("time", descending=True)
    .with_columns(pl.col("time").dt.to_string("%H:%M"))
    .head(100)
    .rename(
        mapping={
            "date": "Date",
            "time": "Time",
            "sport": "Sport",
            "title": "Title",
            "distance": "Distance",
            "duration": "Duration",
            "hrMax": "Max",
            "kcal": "kCal",
            # "vo2max": "VO2max (ml/kg/min)",
            "rpe": "RPE",
            "zoneDistributionPace": "Pace",
            "elevationUp": "Up",
            "elevationDown": "Down",
            "hrAvg": "Avg",
            "zoneDistributionHr": "HR",
        }
    )
    .with_columns(pl.col("Avg").round(0).cast(pl.Int64))
    .with_columns(
        suffix=pl.when(pl.col("Title").str.len_chars() > 15)
        .then(pl.lit("..."))
        .otherwise(pl.lit("")),
    )
    .with_columns(pl.col("Title").str.slice(0, 15) + pl.col("suffix"))
    .drop("suffix")
    .to_pandas()
    .assign(
        HR=lambda d: d["HR"].apply(
            lambda x: [int(_) for _ in x] if x is not None else []
        )
    )
    .assign(
        Pace=lambda d: d["Pace"].apply(
            lambda x: [int(_) for _ in x] if x is not None else []
        )
    )
    .assign(
        # drop the timestamp from Date column
        Date=lambda d: d["Date"].astype(str).str.split(" ").str[0]
    )
)
df

,Date,Time,Sport,Title,Distance,Duration,Up,Down,Avg,Max,kCal,RPE,Pace,HR
0,2025-10-29,08:08,5,Strength,NaN,40m4s,NaN,NaN,87.0,123.0,186,NaN,[],"[88, 12, 0, 0, 0]"
1,2025-10-28,17:01,5,Strength,NaN,48m26s,NaN,NaN,75.0,107.0,151,NaN,[],"[99, 1, 0, 0, 0]"
2,2025-10-28,08:39,4,Treadmill 4x 2k...,18.01,1h25m45s,NaN,NaN,148.0,182.0,1027,NaN,"[14, 2, 11, 27, 15, 1, 5, 26, 0]","[0, 46, 22, 31, 1]"
3,2025-10-27,15:39,4,Bonn Running,16.39,1h28m16s,23.0,42.0,122.0,137.0,1074,11.0,"[1, 3, 27, 61, 7, 1, 0, 0, 0]","[5, 95, 0, 0, 0]"
4,2025-10-27,08:10,5,Strength,NaN,43m29s,NaN,NaN,92.0,129.0,232,NaN,[],"[73, 27, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-09-16,13:21,1,,1.71,5m32s,1.0,2.0,NaN,NaN,71,NaN,"[29, 10, 22, 31, 8, 0, 0]",[]
96,2025-09-16,12:52,1,,2.03,6m23s,8.0,0.0,NaN,NaN,81,NaN,"[21, 11, 31, 16, 20, 0, 0]",[]
97,2025-09-16,08:17,4,"5x1 @3:45, 500@...",18.99,1h28m34s,NaN,NaN,150.0,183.0,1136,NaN,"[12, 12, 15, 19, 3, 4, 12, 22, 0]","[0, 31, 34, 33, 1]"
98,2025-09-15,17:33,5,Strength,NaN,55m10s,NaN,NaN,90.0,147.0,267,NaN,[],"[71, 27, 1, 0, 0]"


In [18]:
(
    GT(df)
    .cols_hide("Sport")
    .tab_stub(rowname_col="Time", groupname_col="Date")
    .fmt_number(columns="Distance", pattern="{x}km", decimals=1)
    .fmt_number(columns=["Avg", "Max", "Up", "Down"], decimals=0)
    .tab_spanner(label="Elevation (m)", columns=["Up", "Down"])
    .tab_spanner(label="Heart Rate (bpm)", columns=["Max", "Avg"])
    .tab_spanner(label="Zones", columns=["Pace", "HR"])
    .tab_options(
        container_width="100%",
        container_height="auto",
        row_group_background_color="WhiteSmoke",
        table_font_names=system_fonts(name="system-ui"),
    )
    .opt_vertical_padding(scale=0.1)
    .cols_width(cases={"Title": "200px"})
    # .tab_style(
    #     style=style.fill(color="Azure"),
    #     locations=loc.body(
    #         columns="Title",
    #         rows=lambda d: (d["Sport"] == 4),
    #     ),
    # )
    .pipe(
        gte.gt_plt_donut,
        columns=["RPE"],
        size=40,
        fill="black",
        domain=[0, 20],
    )
    .pipe(
        gte.gt_plt_bar_stack,
        column="HR",
        palette="Reds",
        width=150,
        spacing=0,
        include_labels=False,
    )
    .pipe(
        gte.gt_plt_bar_stack,
        column="Pace",
        palette="Purples",
        width=150,
        spacing=0,
        include_labels=False,
    )
    .tab_options(data_row_padding="0px")
    .pipe(
        gte.gt_color_box,
        columns=["Max"],
        palette=["white", "red"],
        domain=[80, 200],
        # decimals=0,
    )
    .pipe(
        gte.gt_color_box,
        columns=["Distance"],
        palette="Oranges",
        domain=[0, 30],
    )
    .save("table.png", scale=3)
)

/Users/thomascamminady/Dev/greatest_running_table/.venv/lib/python3.11/site-packages/gt_extras/_utils_column.py:134: UserWarning: Value 30.95 in column 'Distance' is greater than the domain maximum 30. Setting to 30.
  warnings.warn(


GT(_tbl_data=          Date   Time  Sport               Title  Distance  Duration    Up  \
0   2025-10-29  08:08      5            Strength       NaN     40m4s   NaN   
1   2025-10-28  17:01      5            Strength       NaN    48m26s   NaN   
2   2025-10-28  08:39      4  Treadmill 4x 2k...     18.01  1h25m45s   NaN   
3   2025-10-27  15:39      4        Bonn Running     16.39  1h28m16s  23.0   
4   2025-10-27  08:10      5            Strength       NaN    43m29s   NaN   
..         ...    ...    ...                 ...       ...       ...   ...   
95  2025-09-16  13:21      1                          1.71     5m32s   1.0   
96  2025-09-16  12:52      1                          2.03     6m23s   8.0   
97  2025-09-16  08:17      4  5x1 @3:45, 500@...     18.99  1h28m34s   NaN   
98  2025-09-15  17:33      5            Strength       NaN    55m10s   NaN   
99  2025-09-15  17:19      1                          0.95     4m35s   4.0   

    Down    Avg    Max  kCal   RPE                               Pace  \
0    NaN   87.0  123.0   186   NaN                                 []   
1    NaN   75.0  107.0   151   NaN                                 []   
2    NaN  148.0  182.0  1027   NaN   [14, 2, 11, 27, 15, 1, 5, 26, 0]   
3   42.0  122.0  137.0  1074  11.0      [1, 3, 27, 61, 7, 1, 0, 0, 0]   
4    NaN   92.0  129.0   232   NaN                                 []   
..   ...    ...    ...   ...   ...                                ...   
95   2.0    NaN    NaN    71   NaN          [29, 10, 22, 31, 8, 0, 0]   
96   0.0    NaN    NaN    81   NaN         [21, 11, 31, 16, 20, 0, 0]   
97   NaN  150.0  183.0  1136   NaN  [12, 12, 15, 19, 3, 4, 12, 22, 0]   
98   NaN   90.0  147.0   267   NaN                                 []   
99   4.0    NaN    NaN    58   NaN            [90, 10, 0, 0, 0, 0, 0]   

                    HR  
0    [88, 12, 0, 0, 0]  
1     [99, 1, 0, 0, 0]  
2   [0, 46, 22, 31, 1]  
3     [5, 95, 0, 0, 0]  
4    [73, 27, 0, 0, 0]  
..                 ...  
95                  []  
96                  []  
97  [0, 31, 34, 33, 1]  
98   [71, 27, 1, 0, 0]  
99                  []  

[100 rows x 14 columns], _body=<great_tables._gt_data.Body object at 0x12c2fd610>, _boxhead=Boxhead([ColInfo(var='Date', type=<ColInfoTypeEnum.row_group: 3>, column_label='Date', column_align='right', column_width=None), ColInfo(var='Time', type=<ColInfoTypeEnum.stub: 2>, column_label='Time', column_align='right', column_width=None), ColInfo(var='Sport', type=<ColInfoTypeEnum.hidden: 4>, column_label='Sport', column_align='right', column_width=None), ColInfo(var='Title', type=<ColInfoTypeEnum.default: 1>, column_label='Title', column_align='left', column_width='200px'), ColInfo(var='Distance', type=<ColInfoTypeEnum.default: 1>, column_label='Distance', column_align='right', column_width=None), ColInfo(var='Duration', type=<ColInfoTypeEnum.default: 1>, column_label='Duration', column_align='left', column_width=None), ColInfo(var='Up', type=<ColInfoTypeEnum.default: 1>, column_label='Up', column_align='right', column_width=None), ColInfo(var='Down', type=<ColInfoTypeEnum.default: 1>, column_label='Down', column_align='right', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='right', column_width=None), ColInfo(var='Avg', type=<ColInfoTypeEnum.default: 1>, column_label='Avg', column_align='right', column_width=None), ColInfo(var='kCal', type=<ColInfoTypeEnum.default: 1>, column_label='kCal', column_align='right', column_width=None), ColInfo(var='RPE', type=<ColInfoTypeEnum.default: 1>, column_label='RPE', column_align='right', column_width=None), ColInfo(var='Pace', type=<ColInfoTypeEnum.default: 1>, column_label='Pace', column_align='right', column_width=None), ColInfo(var='HR', type=<ColInfoTypeEnum.default: 1>, column_label='HR', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x12c2fe910>, _spanners=Spanners([SpannerInfo(spanner_id='Elevation 